<a href="https://colab.research.google.com/github/ap-nlp-research/language_translation_en_ru_tf2/blob/master/MachineTranslationModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation Project


The goal of the project is to compare the strength of the following recurrent models:

1. Embedded GRU
2. Embedded Bidirectional GRU
3. Embedded GRU encoder-decoder model
4. Embedded GRU encoder-decoder model with Multiplicative Attention

The models implemented in Tensorflow 2.0 with Keras as a high-level API. Models are trained and analyzed based on EN-RU [wmt19_translate dataset](https://www.tensorflow.org/datasets/datasets#wmt19_translate) ([ACL 2019 FOURTH CONFERENCE ON MACHINE TRANSLATION (WMT19)](http://www.statmt.org/wmt19/translation-task.html)).

In [52]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os
import subprocess
import re
import numpy as np
from functools import partial
import tensorflow as tf
from tensorflow import keras

## Data ETL

The data load, extraction, and transformation is done with data_etl() method. This method returns a dictionary containing source data stored under 'x' label. Target data is stored under 'y' label. In addition to the source and target data, the dictionary contains x and y tockenizers (stored as 'x_tk' and 'y_tk'):

In [0]:
def data_etl(download_dir: str = ".", file_name: str = "en_ru.tgz", n_lines: int = 3000,
             lenght_lim_lower: int = 4, length_lim_upper: int = None, num_words: int = 5000) -> dict:
    path = os.path.join(download_dir, file_name)
    en_path = os.path.join(download_dir, "paracrawl-release1.en-ru.zipporah0-dedup-clean.en")
    ru_path = os.path.join(download_dir, "paracrawl-release1.en-ru.zipporah0-dedup-clean.ru")

    print("Start data ETL")

    if os.path.isfile(path):
        print("Reuse pre-downloaded " + path)
        # extract
        subprocess.run("tar --extract --file".split() + [path])
    else:
        # download a dataset
        print("Start downloading")
        completed = subprocess.run(
                ["curl",
                 "https://s3.amazonaws.com/web-language-models/paracrawl/release1/paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz",
                 "--output", 
                 path]
        )
        if completed.check_returncode() is not None:
            print("Downloading returned error code:", completed.check_returncode())
    # extract
    if not (os.path.isfile(en_path) and os.path.isfile(ru_path)):
        subprocess.run("tar --extract --file".split() + [path])
    print("Data downloaded and extracted")

    # read data
    en = read_lines(en_path, n_lines)
    ru = read_lines(ru_path, n_lines)

    print("Extracted successfully")
    # slit numbers
    en = [re.sub("[0-9]", " \g<0>", s) for s in en]
    ru = [re.sub("[0-9]", " \g<0>", s) for s in ru]
    
    if length_lim_upper is None:
      # remove extra long sequence as they will skew the loss towards 0
      lengths = np.array([len(s.split()) for s in en])
      l_mean = lengths.mean()
      l_std = lengths.mean()
      length_lim_upper = int(l_mean + l_std*2)
      

    # filter out small samples
    indices = list(filter(lambda idx: length_lim_upper > len(en[idx].split()) >=lenght_lim_lower, range(len(en))))
    en = [en[idx] for idx in indices]
    ru = [ru[idx] for idx in indices]

    # Tockenize
    x, x_tk = tokenize(en, num_words=num_words, filters_regex=None)
    y, y_tk = tokenize(ru, num_words=num_words * 3, filters_regex=None)

    x = pad(x)
    y_length = max([len(sentence) for sentence in y])
    y = pad(y, length=x.shape[1] if x.shape[1]>y_length else y_length)

    print("Transformed successfully")

    return {'x': x, 'y': y, 'x_tk': x_tk, 'y_tk': y_tk}


def read_lines(from_file: str, n_lines: int) -> list:
    if not os.path.isfile(from_file):
        raise FileNotFoundError(from_file)

    lines: list = []
    with open(from_file, "r") as f:
        for idx, line in enumerate(f):
            if idx < n_lines:
                lines.append(line)
            else:
                break

    return lines


def tokenize(x: list, num_words: int = 5000, filters_regex: str = None):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :param num_words: maximum number of words in the vocabulary - rare words will be replaced with a [rare] token
    :param filters_regex: regular expression filter string such as [^a-zA-Z0-9/-]
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    if filters_regex is not None:
        x_tk = keras.preprocessing.text.Tokenizer(num_words=num_words, filters=filters_regex, oov_token="[rare]")
    else:
        x_tk = keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token="[rare]")

    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


def pad(x, length=None) -> np.ndarray:
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])

    return keras.preprocessing.sequence.pad_sequences(x, maxlen=length, padding='post')

dataset = data_etl()

{
    'x': np.ndarray,
    'y': np.ndarray,
    'x_tk': keras.preprocessing.text.Tokenizer,
    'y_tk': keras.preprocessing.text.Tokenizer
}

In [118]:
dataset = data_etl(download_dir=".", n_lines=5000, num_words=3000)

Start data ETL
Reuse pre-downloaded ./en_ru.tgz
Data downloaded and extracted
Extracted successfully
Transformed successfully


## Utility Functions

In addition to the data ETL, the code below provides two additional functions for converting logits into word indicies and converting word indicies into text.

In [0]:
def logits_to_id(logits):
    """
    Turns logits into word ids
    :param logits: Logits from a neural network
    """
    return np.argmax(logits, 1)

def id_to_text(idx, tokenizer):
    """
    Turns id into text using the tokenizer
    :param idx: word id
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in idx]).replace(" <PAD>", "")

In [119]:
print("Here is an example for a samples number 1:")
print("Source('en') example:", id_to_text(dataset['x'][0], dataset['x_tk']))
print("Target('ru') example:", id_to_text(dataset['y'][0], dataset['y_tk']))
print(" ")
print("Samples number 2:")
print("Source('en') example:", id_to_text(dataset['x'][-10], dataset['x_tk']))
print("Target('ru') example:", id_to_text(dataset['y'][-10], dataset['y_tk']))
print("source vocabulary size:", dataset['x'].max())
print("target vocabulary size:", dataset['y'].max())
print("Source shape:", dataset['x'].shape)
print("Target shape:", dataset['y'].shape)

Here is an example for a samples number 1:
Source('en') example: the company has been registered with the [rare] court in prague in section b file 1 4 8 5 7
Target('ru') example: фирма зарегистрирована в городском суде в г праге раздел б вкладыш 1 4 8 5 7
 
Samples number 2:
Source('en') example: pontoon floating platforms are quick to [rare] resistant to [rare] maintenance free and a good value and safe alternative to traditional wooden platforms
Target('ru') example: понтонные плавучие пирсы [rare] [rare] установки [rare] к природным воздействиям [rare] и [rare] эксплуатации являются [rare] и безопасной [rare] традиционным деревянным [rare]
source vocabulary size: 2999
target vocabulary size: 8999
Source shape: (3602, 65)
Target shape: (3602, 99)


## Models

The models are implemented with a similar set of parameters. The main idea is to keep models as small and simple as possible to quickly train them and validate the difference the primarely derived from model architectures. The summary of main hyper parameters presented below:

* Mapping:
    - Embeddings - word indices will be mapped into a 16-dimentional space
    - Dense mapping - recurrence outputs mapped into the target-language space, represented with OHE, via Dense mapping
* Layers:
    - GRU - number of units 256
    - Bidirectional GRU - number of untis set up to 128 in order to keep the total number of units the same (256)
    - Batch Normalization - To speed up the training batch normalization is inserted after embeddings and before dense mapping
* Optimization:
    - Adam - all models trained with Adam optimizer and the same learning rate (1e-3)
* Loss function:
    - sparse_categorical_crossentropy_from_logits - keras.losses.sparse_categorical_crossentropy

In [0]:
learning_rate = 1e-3
embeddings_units = 32
gru_units = 256
epochs = 60
validation_split = 0.1
batch_size = 256
loss = keras.losses.sparse_categorical_crossentropy

**Model list:**

1. Embedded GRU
2. Embedded Bidirectional GRU
3. Embedded GRU encoder-decoder model
4. Embedded GRU encoder-decoder model with Multiplicative Attention

#### Model 1 - Embedded GRU

In [128]:
def embedded_gru_model(input_shape, output_sequence_length, source_vocab_size, target_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_seq = keras.Input(input_shape[1:])
    if output_sequence_length>input_shape[1]:
        expanded_seq = keras.backend.squeeze(
            keras.layers.ZeroPadding1D((0, output_sequence_length-input_shape[1]))(
                keras.layers.Reshape((input_shape[1], 1))(input_seq)
            ),
            axis = -1
        )
    else:
        expanded_seq = input_seq
    embedded_seq = keras.layers.TimeDistributed(keras.layers.BatchNormalization(axis=-1))(
        keras.layers.Embedding(source_vocab_size, embeddings_units, input_length=output_sequence_length)(expanded_seq)
    )
    rnn = keras.layers.TimeDistributed(keras.layers.BatchNormalization(axis=-1))(
        keras.layers.GRU(int(gru_units), return_sequences=True)(embedded_seq)
    )
    logits = keras.layers.TimeDistributed(keras.layers.BatchNormalization(axis=-1))(
        keras.layers.TimeDistributed(keras.layers.Dense(4*gru_units, activation='elu'))(rnn)
    )
    probabilities = keras.layers.TimeDistributed(keras.layers.Dense(target_vocab_size, activation='softmax'))(logits)
    
    model = keras.Model(input_seq, probabilities)
    
    model.compile(loss=loss,
                  optimizer=keras.optimizers.Adam(learning_rate, clipnorm=3.0),
                  metrics=['accuracy'])
    return model

  
# Train the neural network
embed_rnn_model = embedded_gru_model(
    dataset['x'].shape,
    dataset['y'].shape[1],
    dataset['x'].max()+1,
    dataset['y'].max()+1)



print("Model summary:")
embed_rnn_model.summary()

Model summary:
Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 65)]              0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 65, 1)             0         
_________________________________________________________________
zero_padding1d_3 (ZeroPaddin (None, 99, 1)             0         
_________________________________________________________________
Squeeze_3 (TensorFlowOpLayer [(None, 99)]              0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 99, 32)            96000     
_________________________________________________________________
time_distributed_72 (TimeDis (None, 99, 32)            128       
_________________________________________________________________
unified_gru_15 (UnifiedGRU)  (None, 99, 256

In [129]:
embed_rnn_model.fit(
    dataset['x'], 
    dataset['y'][:,:, None], 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split=validation_split)

Train on 3241 samples, validate on 361 samples
Epoch 1/50
3241/3241 [==============================] - 10s 3ms/sample - loss: 4.3589 - accuracy: 0.7318 - val_loss: 7.5388 - val_accuracy: 0.8467
Epoch 2/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.6856 - accuracy: 0.8346 - val_loss: 6.5262 - val_accuracy: 0.8467
Epoch 3/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.5374 - accuracy: 0.8464 - val_loss: 6.4239 - val_accuracy: 0.8467
Epoch 4/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.4532 - accuracy: 0.8511 - val_loss: 6.8451 - val_accuracy: 0.8467
Epoch 5/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.3640 - accuracy: 0.8552 - val_loss: 7.1115 - val_accuracy: 0.8467
Epoch 6/50
3241/3241 [==============================] - 8s 3ms/sample - loss: 1.2717 - accuracy: 0.8593 - val_loss: 7.3468 - val_accuracy: 0.8462
Epoch 7/50
3241/3241 [==============================] - 8s 3ms/sample - loss

In [130]:
# Print prediction(s)
sentense_id = -10
x_sample = dataset['x'][sentense_id]
y_sample = dataset['y'][sentense_id]
print("Source('en') example:", id_to_text( x_sample, dataset['x_tk'] ))
print("Source('ru') example:", id_to_text( y_sample, dataset['y_tk'] ))
prediction = embed_rnn_model.predict(x_sample[None, :], verbose=1).squeeze()
print("Translation(en_ru) example:", id_to_text( logits_to_id(prediction), dataset['y_tk'] ))

Source('en') example: pontoon floating platforms are quick to [rare] resistant to [rare] maintenance free and a good value and safe alternative to traditional wooden platforms
Source('ru') example: понтонные плавучие пирсы [rare] [rare] установки [rare] к природным воздействиям [rare] и [rare] эксплуатации являются [rare] и безопасной [rare] традиционным деревянным [rare]
1/1 [==============================] - 0s 80ms/sample
Translation(en_ru) example: мы его и пожалуйста не с [rare] в в а в подачи подачи о великого своим данные которая в в так животные и


#### Model 2 - BiDirectional GRU

In [132]:
def bd_gru_model(input_shape, output_sequence_length, source_vocab_size, target_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_seq = keras.Input(input_shape[1:])
    if output_sequence_length>input_shape[1]:
        expanded_seq = keras.backend.squeeze(
            keras.layers.ZeroPadding1D((0, output_sequence_length-input_shape[1]))(
                keras.layers.Reshape((input_shape[1], 1))(input_seq)
            ),
            axis = -1
        )
    else:
        expanded_seq = input_seq
    embedded_seq = keras.layers.TimeDistributed(keras.layers.BatchNormalization(axis=-1))(
        keras.layers.Embedding(source_vocab_size, embeddings_units, input_length=output_sequence_length)(expanded_seq)
    )
    rnn = keras.layers.TimeDistributed(keras.layers.BatchNormalization(axis=-1))(
        keras.layers.Bidirectional(keras.layers.GRU(int(gru_units/2), return_sequences=True))(embedded_seq)
    )
    logits = keras.layers.TimeDistributed(keras.layers.BatchNormalization(axis=-1))(
        keras.layers.TimeDistributed(keras.layers.Dense(4*gru_units, activation='elu'))(rnn)
    )
    probabilities = keras.layers.TimeDistributed(keras.layers.Dense(target_vocab_size, activation='softmax'))(logits)
    
    model = keras.Model(input_seq, probabilities)
    
    model.compile(loss=loss,
                  optimizer=keras.optimizers.Adam(learning_rate, clipnorm=3.0),
                  metrics=['accuracy'])
    return model

  
# Train the neural network
bd_rnn_model = embedded_gru_model(
    dataset['x'].shape,
    dataset['y'].shape[1],
    dataset['x'].max()+1,
    dataset['y'].max()+1)



print("Model summary:")
bd_rnn_model.summary()

Model summary:
Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 65)]              0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 65, 1)             0         
_________________________________________________________________
zero_padding1d_5 (ZeroPaddin (None, 99, 1)             0         
_________________________________________________________________
Squeeze_5 (TensorFlowOpLayer [(None, 99)]              0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 99, 32)            96000     
_________________________________________________________________
time_distributed_82 (TimeDis (None, 99, 32)            128       
_________________________________________________________________
unified_gru_17 (UnifiedGRU)  (None, 99, 256

In [133]:
bd_rnn_model.fit(
    dataset['x'], 
    dataset['y'][:,:, None], 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split=validation_split
)

Train on 3241 samples, validate on 361 samples
Epoch 1/50
3241/3241 [==============================] - 10s 3ms/sample - loss: 4.3500 - accuracy: 0.7389 - val_loss: 7.6888 - val_accuracy: 0.8467
Epoch 2/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.6648 - accuracy: 0.8370 - val_loss: 6.7278 - val_accuracy: 0.8467
Epoch 3/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.5327 - accuracy: 0.8471 - val_loss: 6.4699 - val_accuracy: 0.8467
Epoch 4/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.4463 - accuracy: 0.8511 - val_loss: 6.7554 - val_accuracy: 0.8467
Epoch 5/50
3241/3241 [==============================] - 9s 3ms/sample - loss: 1.3504 - accuracy: 0.8556 - val_loss: 7.0127 - val_accuracy: 0.8467
Epoch 6/50
3241/3241 [==============================] - 8s 3ms/sample - loss: 1.2524 - accuracy: 0.8602 - val_loss: 7.2343 - val_accuracy: 0.8467
Epoch 7/50
3241/3241 [==============================] - 8s 3ms/sample - loss

In [134]:
# Print prediction(s)
sentense_id = -10
x_sample = dataset['x'][sentense_id]
y_sample = dataset['y'][sentense_id]
print("Source('en') example:", id_to_text( x_sample, dataset['x_tk'] ))
print("Source('ru') example:", id_to_text( y_sample, dataset['y_tk'] ))
prediction = bd_rnn_model.predict(x_sample[None, :], verbose=1).squeeze()
print("Translation(en_ru) example:", id_to_text( logits_to_id(prediction), dataset['y_tk'] ))

Source('en') example: pontoon floating platforms are quick to [rare] resistant to [rare] maintenance free and a good value and safe alternative to traditional wooden platforms
Source('ru') example: понтонные плавучие пирсы [rare] [rare] установки [rare] к природным воздействиям [rare] и [rare] эксплуатации являются [rare] и безопасной [rare] традиционным деревянным [rare]
1/1 [==============================] - 0s 80ms/sample
Translation(en_ru) example: в 1 2 на к to на в to в этого в а на и они и интернет св и и по и конечного надежду ты столярным столярным столярным
